# Micro-homology endjoining analysis

In [33]:

from pydna.dseqrecord import Dseqrecord
from constrain.lab.PCR import primer_tm_neb
from pydna.design import primer_design
from pydna.dseqrecord import Dseqrecord
from  Bio.SeqRecord import SeqRecord
from pydna.dseqrecord import Dseqrecord
from typing import List

from typing import List

from Bio import SeqIO 
from pydna.dseqrecord import Dseqrecord
import pandas as pd
from constrain.lab.PCR import primer_tm_neb
from pydna.design import primer_design
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO 
from pydna.dseqrecord import Dseqrecord
from Bio.SeqFeature import SeqFeature, FeatureLocation
import pandas as pd
from pydna.assembly import Assembly
from  Bio.SeqRecord import SeqRecord
from dna_features_viewer import GraphicFeature, CircularGraphicRecord
from dna_features_viewer import BiopythonTranslator
import os
from Bio import SeqIO 

from Bio.Seq import Seq


In [42]:
from teemi.design.fetch_sequences import read_fasta_files
promoter = read_fasta_files('../data/parts/pTEF.fasta')[0][-50:]
RFP = read_fasta_files('../data/parts/RFP.fasta')[0][:50]

In [43]:
print(promoter[-50:].seq)
print(RFP[:50].reverse_complement().seq)

CTGAGCACTTCTCCCTTTTATATTCCACAAAACATAACACAACCTTCACC
ATGCGCACCTTGAAGCGCATGAACTCCTTGATGACGTCCTCGGAGGAGGC


In [44]:
def generate_kmers(sequence, k):
    """
    Generate all k-mers from a given sequence.

    Parameters:
    - sequence (str): Input sequence.
    - k (int): Length of the k-mers to generate.

    Returns:
    - list: List of generated k-mers.

    Raises:
    - ValueError: If k is less than or equal to 0 or greater than the length of the sequence.
    """
    kmers = []
    n = len(sequence)

    if k <= 0 or k > n:
        return kmers

    for i in range(n - k + 1):
        kmer = sequence[i:i+k]
        kmers.append(kmer)

    return kmers


import pandas as pd
from Bio.SeqRecord import SeqRecord

def micro_homolgy(sequence1: SeqRecord, sequence2: SeqRecord, k: int = 3) -> pd.DataFrame:
    """
    Count the occurrences of shared k-mers between two sequences and return the results as a DataFrame.

    Parameters:
    - sequence1 (SeqRecord): First input sequence as a SeqRecord object.
    - sequence2 (SeqRecord): Second input sequence as a SeqRecord object.
    - k (int): Length of the k-mers to consider (default: 3).

    Returns:
    - pd.DataFrame: DataFrame with columns 'Sequence' and 'Count', sorted by count in descending order.

    Notes:
    - The function uses the generate_kmers function internally to extract k-mers from the input sequences.

    Examples:
    >>> from Bio import SeqIO
    >>> sequence1 = SeqIO.read("sequence1.fasta", "fasta")
    >>> sequence2 = SeqIO.read("sequence2.fasta", "fasta")
    >>> result = micro_homolgy(sequence1, sequence2, k=3)
    >>> print(result)
      Sequence  Count
    0      TCG      3
    1      GAT      3
    2      CGA      3
    """
    kmer1 = generate_kmers(sequence1.seq, k)
    sequence2 = sequence2.reverse_complement()
    kmer2 = generate_kmers(sequence2.seq, k)

    kmer_counts = {}

    for seq1 in kmer1:
        for seq2 in kmer2:
            if seq1 == seq2:
                if seq1 in kmer_counts:
                    kmer_counts[seq1] += 1
                else:
                    kmer_counts[seq1] = 1

    sorted_kmers = sorted(kmer_counts.items(), key=lambda x: x[1], reverse=True)
    df = pd.DataFrame(sorted_kmers, columns=['Sequence', 'Count'])

    return df


In [48]:
counts = micro_homolgy(promoter, RFP, k=3)
counts

,Sequence,Count
0,"(C, T, T)",6
1,"(C, C, T)",6
2,"(T, G, A)",4
3,"(C, A, C)",4
4,"(T, C, C)",4
5,"(A, A, C)",3
6,"(G, A, G)",2
7,"(G, C, A)",2
8,"(C, T, C)",2
9,"(A, C, C)",2


In [51]:
total = counts['Count'].sum()

In [52]:
total

38